In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
from defs import ArgoverseDataset
from defs import my_collate

### Create a dataset class 

### Create a loader to enable batch processing

### Visualize the batch of sequences

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(4, 64, 3),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            # torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(64, 64*2, 3),
            torch.nn.BatchNorm2d(64*2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Flatten(),
            torch.nn.Linear(25088, 10028), # 18240
            torch.nn.Dropout(0.3), # 0.5
            torch.nn.ReLU(),
            torch.nn.Linear(10028, 5000), # 18240
            torch.nn.Dropout(0.2), # 0.5
            torch.nn.ReLU(),
            torch.nn.Linear(5000, 3600),
            torch.nn.Dropout(0.1), # 0.2
            torch.nn.ReLU()
        )
        
    def forward(self, x):
        
        x = self.model(x)
        return x

In [ ]:
from tqdm import tqdm_notebook as tqdm

def train(model, train_loader, device, optimizer, epoch, log_interval=10000):
    model.train()
    criterion = nn.MSELoss()
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    # for i_batch, sample_batch in enumerate(train_loader):
    for batch_idx, (inp, out) in enumerate(iterator):
         
        # inp, out = sample_batch
        inp = inp.to(device)
        out = out.to(device)
        
        optimizer.zero_grad()
        
        inp = inp.reshape(inp.shape[0], inp.shape[3], inp.shape[1], inp.shape[2])
        pred_out = model(inp)

        pred_out = pred_out.reshape(out.shape[0], out.shape[1] * out.shape[2], out.shape[3])
        out = out.reshape(out.shape[0], out.shape[1] * out.shape[2], out.shape[3])
        
        loss = torch.sqrt(criterion(pred_out, out))
        loss.backward()
        optimizer.step()
        
        counter += 1
        iterator.set_postfix(loss=(loss.item()*inp.size(0) / (counter * train_loader.batch_size)))

In [ ]:
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(test_loader):
            inp, out = sample_batch
            inp = inp.to(device)
            out = out.to(device)
            
            inp = inp.reshape(inp.shape[0], inp.shape[3], inp.shape[1], inp.shape[2])
            pred_out = model(inp)
            
            pred_out = pred_out.reshape(out.shape[0], out.shape[1] * out.shape[2], out.shape[3])
            out = out.reshape(out.shape[0], out.shape[1] * out.shape[2], out.shape[3])
            
            test_loss += torch.sqrt(criterion(pred_out, out)).item() # sum up batch loss
            
    test_loss /= len(test_loader.dataset)

In [ ]:
if __name__ == '__main__':
    """Change to the data folder"""
    train_path = "./new_train/new_train"
    test_path = "./new_val_in/new_val_in"
    # number of sequences in each dataset
    # train:205942  val:3200 test: 36272 
    # sequences sampled at 10HZ rate
    
    # intialize a dataset
    val_dataset  = ArgoverseDataset(data_path=train_path)
    test_dataset = ArgoverseDataset(data_path=test_path)

    TRAIN_SET, TEST_SET = torch.utils.data.random_split(val_dataset, [164753, 41189])

    batch_size_train = 16
    batch_size_test = 1024


    train_loader = DataLoader(TRAIN_SET,batch_size=batch_size_train, shuffle = True, collate_fn=my_collate, num_workers=2,pin_memory=True)
    test_loader = DataLoader(TEST_SET,batch_size=batch_size_test, shuffle = True, collate_fn=my_collate, num_workers=2, pin_memory=True)

    len(train_loader)

    learning_rate = 0.01
    momentum = 0.5
    device = "cuda"
    model = CNN().cuda() #using cpu here
    torch.backends.cudnn.benchmark = True
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=momentum)
    num_epoch = 10

    for epoch in range(1, num_epoch + 1):
            train(model, train_loader, device, optimizer, epoch)
            test(model, test_loader, device)

# Submission

In [ ]:
def collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    return inp

t_loader = DataLoader(test_dataset,batch_size=1, shuffle = False, collate_fn=collate, num_workers=0)

In [ ]:
import csv

header = ['ID']
for i in range(1, 61):
    header.append('v' + str(i))
    
with open('sample_submission.csv', 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(header) 
        
    for i_batch, sample_batch in enumerate(t_loader):
        header = []
        header.append(test_dataset[i_batch]['scene_idx'])
        
        model.eval()
        inp = sample_batch
        inp = inp.reshape(inp.shape[0], inp.shape[3], inp.shape[1], inp.shape[2])
        inp = inp.to(device)
        pred_out = model(inp)
        pred_out = pred_out.reshape(1, 60, 30, 2)
        pred_out = pred_out.squeeze() 
        track_id = test_dataset[i_batch]['track_id']
        track_id = track_id[:,0,0]
        index = 0
        for i in range(len(track_id)):
            if test_dataset[i_batch]['agent_id'] == track_id[i]:
                index = i
                break
        p_out = pred_out[index]
        p_out = p_out.reshape(30*2)
        for i in range(len(p_out)):
            header.append(p_out[i].item())
            
        csvwriter.writerow(header)

In [ ]:
import matplotlib.pyplot as plt
import random

TRAIN_SET, TEST_SET = torch.utils.data.random_split(val_dataset, [164753, 41189])

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    """TODO:
      Deep learning model
      training routine
    """
    show_sample_batch(sample_batch, agent_id)
    break